# Lab1: Gaussian Elimination

In this section we define some Python functions to help us solve linear systems in the most direct way.  The algorithm is known as Gaussian Elimination, which we will simply refer to as **elimination** from this point forward.  The idea of elimination is to exchange the system we are given with another system that has the same solution, but is much easier to solve.  To this end we will perform a series of steps called **row operations** which preserve the solution of the system while gradually making the solution more accessible.  There are three such operations we may perform.
1. Exchange the position of two equations.
2. Multiply an equation by any nonzero number.
3. Replace any equation with the sum of itself and a multiple of another equation.



<a id='GE1'></a>
### Example 1:  Row operations and elimination

Let's look at an example.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2 
\end{align*}
$$

We could swap the first and last equation,

$$
\begin{align*}
4x_1 + 2x_2 -3x_3 & = & -2 \\
2x_1 + x_2 + 8x_3 & = & 18\\
x_1 - x_2 + x_3 & = & 3
\end{align*}
$$

or we could multiply the first equation by $5$,

$$
\begin{align*}
5x_1 - 5x_2 + 5x_3 & = & 15\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2 
\end{align*}
$$

or we could add 2 times the first equation to the last equation.


$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 3x_3 & = & 8\\
6x_1 \quad\quad -x_3 & = & 4 
\end{align*}
$$

The last operation is the most important because it allows us to *eliminate* a variable from one of the equations.  Note that the third equation no longer contains the $x_2$ term.  This is the key to the elimination algorithm.

For computational purposes we can dispense with the variable names and the "=" symbol, and arrange all of the actual numbers in an array.

$$
\begin{equation}
\left[ \begin{array}{rrrr} 1 & -1 & 1 & 3 \\ 2 & 1 & 8 & 18 \\ 4 & 2 & -3 & -2 \end{array}\right]
\end{equation}
$$

Now let's build a NumPy array with these values.  We'll assign the array the name $\texttt{A}$, so that we can refer to it later. 

In [1]:
import numpy as np
A=np.array([[1,-1,1,3],[2,1,8,18],[4,2,-3,-2]])
A

array([[ 1, -1,  1,  3],
       [ 2,  1,  8, 18],
       [ 4,  2, -3, -2]])

We could dive in an start performing operations on our array, but instead we will first write a few bits of code that will do each of these operations individually.  We will tuck each operation inside a Python function so that we can use it again for future calculations.

In [2]:
def RowSwap(A,k,l):
# =============================================================================
#     A is a NumPy array.  RowSwap will return duplicate array with rows
#     k and l swapped.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def RowScale(A,k,scale):
# =============================================================================
#     A is a NumPy array.  RowScale will return duplicate array with the
#     entries of row k multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def RowAdd(A,k,l,scale):
# =============================================================================
#     A is a numpy array.  RowAdd will return duplicate array with row
#     l modifed.  The new values will be the old values of row l added to 
#     the values of row k, multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B



We now have three new functions called $\texttt{RowSwap}$,$\texttt{RowScale}$,and $\texttt{RowAdd}$.  Let's try them out to see what they produce.

In [3]:
print("The original matrix A:")
print(A)

print("\nSwap rows 0 and 2 in matrix A:")
print(RowSwap(A, 0, 2))

print("\nScale row 2 of matrix A by a factor of 0.5:")
print(RowScale(A, 2, 0.5))

print("\nAdd 2 times row 0 of matrix A to row 1:")
print(RowAdd(A, 0, 1, 2))

The original matrix A:
[[ 1 -1  1  3]
 [ 2  1  8 18]
 [ 4  2 -3 -2]]

Swap rows 0 and 2 in matrix A:
[[ 4.  2. -3. -2.]
 [ 2.  1.  8. 18.]
 [ 1. -1.  1.  3.]]

Scale row 2 of matrix A by a factor of 0.5:
[[ 1.  -1.   1.   3. ]
 [ 2.   1.   8.  18. ]
 [ 2.   1.  -1.5 -1. ]]

Add 2 times row 0 of matrix A to row 1:
[[ 1. -1.  1.  3.]
 [ 4. -1. 10. 24.]
 [ 4.  2. -3. -2.]]


The goal of elimination is to perform row operations on this array in order to produce a new array with a structure that looks something like this.

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

*(Note that the * symbols here represent different unknown values that may or may not be 0 or 1.)*

We will carry out the row operations and save our progress as arrays with new names  after each step.  For example, we might name them $\texttt{A1}$, $\texttt{A2}$, $\texttt{A3}$, etc. This way we can check the progress, or go back and make changes to our code if we like.  

In [4]:
# Add -2 times row 0 to row 1
A1 = RowAdd(A, 0, 1, -2)
print("After adding -2 times row 0 to row 1:")
print(A1, '\n')

# Add -4 times row 0 to row 2
A2 = RowAdd(A1, 0, 2, -4)
print("After adding -4 times row 0 to row 2:")
print(A2, '\n')

# Add -2 times row 1 to row 2
A3 = RowAdd(A2, 1, 2, -2)
print("After adding -2 times row 1 to row 2:")
print(A3, '\n')

# Multiply row 1 by 1/3
A4 = RowScale(A3, 1, 1.0 / 3)
print("After multiplying row 1 by 1/3:")
print(A4, '\n')

# Multiply row 2 by -1/19
print("After multiplying row 2 by -1/19:")
print(RowScale(A4, 2, -1.0 / 19))

After adding -2 times row 0 to row 1:
[[ 1. -1.  1.  3.]
 [ 0.  3.  6. 12.]
 [ 4.  2. -3. -2.]] 

After adding -4 times row 0 to row 2:
[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   6.  -7. -14.]] 

After adding -2 times row 1 to row 2:
[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   0. -19. -38.]] 

After multiplying row 1 by 1/3:
[[  1.  -1.   1.   3.]
 [  0.   1.   2.   4.]
 [  0.   0. -19. -38.]] 

After multiplying row 2 by -1/19:
[[ 1. -1.  1.  3.]
 [ 0.  1.  2.  4.]
 [-0. -0.  1.  2.]]


Now let's translate this array back to the description of the system with all the symbols in place.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
x_2 + 2x_3 & = & 4\\
x_3 & = & 2 
\end{align*}
$$

After the steps of elimination, we have what is known as a **upper triangular** system.  The solution to this system can be found without much effort by working backwards from the last equation.  The last equation tells us that $x_3 = 2$.  If we substitute that value into the second equation, we find that $x_2 = 0$.  Finally, if we substitute these values back into the first equation, we find that $x_1 = 1.$  This process for finding the solution of the upper triangular system is called **back substitution**.

### Example 2:  Elimination on a random array

If a system of equations has a solution, the elimination algorithm will always result in a upper triangular system that can be solved by back substitution.  In this next example, we look at the calculation with a small change to see it in a more general way.  This time when we use the $\texttt{RowAdd}$ function, we will set the *scale* parameter based on the values in the array.  

To help us avoid writing the code based on the entries in any specific matrix, we will make up a matrix of random numbers using the $\texttt{random}$ module.

In [5]:
R = np.random.randint(-8,8,size=(3,4))
R

array([[-1,  6,  4,  6],
       [ 6,  5, -1,  2],
       [ 7,  1,  3,  2]], dtype=int32)

In [6]:
# Scale the first row based on the first element in that row
R1 = RowScale(R, 0, 1.0 / R[0][0])
print("After scaling the first row:")
print(R1, '\n')

# Add the first row to the second based on the first element in the second row
R2 = RowAdd(R1, 0, 1, -R[1][0])
print("After adding the first row to the second row:")
print(R2, '\n')

# Add the first row to the last based on the first element in the last row
R3 = RowAdd(R2, 0, 2, -R2[2][0])
print("After adding the first row to the last row:")
print(R3, '\n')

# Scale the second row based on the second element in that row
R4 = RowScale(R3, 1, 1.0 / R3[1][1])
print("After scaling the second row:")
print(R4, '\n')

# Add the second row to the last based on the second element in the last row
R5 = RowAdd(R4, 1, 2, -R4[2][1])
print("After adding the second row to the last row:")
print(R5, '\n')

# Scale the last row based on the last element in that row
R6 = RowScale(R5, 2, 1.0 / R5[2][2])
print("After scaling the last row:")
print(R6)

After scaling the first row:
[[ 1. -6. -4. -6.]
 [ 6.  5. -1.  2.]
 [ 7.  1.  3.  2.]] 

After adding the first row to the second row:
[[ 1. -6. -4. -6.]
 [ 0. 41. 23. 38.]
 [ 7.  1.  3.  2.]] 

After adding the first row to the last row:
[[ 1. -6. -4. -6.]
 [ 0. 41. 23. 38.]
 [ 0. 43. 31. 44.]] 

After scaling the second row:
[[ 1.         -6.         -4.         -6.        ]
 [ 0.          1.          0.56097561  0.92682927]
 [ 0.         43.         31.         44.        ]] 

After adding the second row to the last row:
[[ 1.         -6.         -4.         -6.        ]
 [ 0.          1.          0.56097561  0.92682927]
 [ 0.          0.          6.87804878  4.14634146]] 

After scaling the last row:
[[ 1.         -6.         -4.         -6.        ]
 [ 0.          1.          0.56097561  0.92682927]
 [ 0.          0.          1.          0.60283688]]


Once we understand how the row operations work, and we are sure that they are working correctly, we might not have any reason to store the results of the intermediate steps of elimination.  In that case, we could simplify the code considerably by resusing the same array name for each step.  The following code would produce the same result, but we would no longer have access to the original array, or any of the intermediate steps.

```
R = RowScale(R,0,1.0/R[0][0])
R = RowAdd(R,0,1,-R[1][0])
R = RowAdd(R,0,2,-R[2][0])
R = RowScale(R,1,1.0/R[1][1])
R = RowAdd(R,1,2,-R[2][1])
R = RowScale(R,2,1.0/R[2][2])
print(R)
```


Execute the code in this example several times.  Each time array $\texttt{R}$ is created it will be populated with random integers between -8 and 8.  Does the code always produce a upper triangular system ready for back substitution?  See if you can figure out which part of the process might fail, and why?

### Example 3: Finding pivots

As we can see, the code in the last example fails if a zero shows up as any of the entries in the array that we divide by in order to calculate the scale factors.  These critical entries are known as the **pivots**, and their locations in the matrix are called **pivot positions**.  By definition, pivots must be nonzero.  If a zero arises in a pivot position during the elimination steps, we can try to exchange the order of the rows to move a nonzero entry into the pivot position.  Let's first try this for a specific array before trying to write code that will work for a random array.


$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 \quad\quad -9x_3 & = & 0 
\end{align*}
$$


In [7]:
G = np.array([[1,-1,1,3],[2,-2,4,8],[3,0,-9,0]])
G


array([[ 1, -1,  1,  3],
       [ 2, -2,  4,  8],
       [ 3,  0, -9,  0]])

In [8]:
# Add -2 times row 0 to row 1
G1 = RowAdd(G, 0, 1, -2)
print("After adding -2 times row 0 to row 1:")
print(G1, '\n')

# Add -3 times row 0 to row 2
G2 = RowAdd(G1, 0, 2, -3)
print("After adding -3 times row 0 to row 2:")
print(G2)

After adding -2 times row 0 to row 1:
[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 3.  0. -9.  0.]] 

After adding -3 times row 0 to row 2:
[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   3. -12.  -9.]]


Now there is a zero in the middle pivot position.  We can swap the middle and last equation in order to carry on the elimination.

In [9]:
# Swap rows 1 and 2
G3 = RowSwap(G2, 1, 2)
print("After swapping rows 1 and 2:")
print(G3, '\n')

# Scale the new row 1 by 1/3
G4 = RowScale(G3, 1, 1.0 / 3)
print("After scaling the new row 1 by 1/3:")
print(G4, '\n')

# Scale the new row 2 by 1/2
G5 = RowScale(G4, 2, 1.0 / 2)
print("After scaling the new row 2 by 1/2:")
print(G5)

After swapping rows 1 and 2:
[[  1.  -1.   1.   3.]
 [  0.   3. -12.  -9.]
 [  0.   0.   2.   2.]] 

After scaling the new row 1 by 1/3:
[[ 1. -1.  1.  3.]
 [ 0.  1. -4. -3.]
 [ 0.  0.  2.  2.]] 

After scaling the new row 2 by 1/2:
[[ 1. -1.  1.  3.]
 [ 0.  1. -4. -3.]
 [ 0.  0.  1.  1.]]


We write the system again as a familar set of equations.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
x_2 - 4x_3 & = & -3\\
x_3 & = & 1 
\end{align*}
$$

Applying back substitution, we find that $x_2 = 1$ and $x_1=3$.

It is worth noting that the swapping of rows is only necessary as a matter of organization.  Here is another way that we could have done the elimination and ended up with a system that is just the same.


In [10]:
# Scale row 1 by 1/2
G3_alternative = RowScale(G2, 1, 1.0 / 2)
print("After scaling row 1 by 1/2:")
print(G3_alternative, '\n')

# Scale row 2 by 1/3
G4_alternative = RowScale(G3_alternative, 2, 1.0 / 3)
print("After scaling row 2 by 1/3:")
print(G4_alternative)

After scaling row 1 by 1/2:
[[  1.  -1.   1.   3.]
 [  0.   0.   1.   1.]
 [  0.   3. -12.  -9.]] 

After scaling row 2 by 1/3:
[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  1. -4. -3.]]


The array produced represents the same simplified system, with the equations in a different order of course.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_2 - 4x_3 & = & -3
\end{align*}
$$

The idea of back substitution works just as well with this form of the system, but the organization of the algorithm becomes slightly more complicated.

<a id='GE4'></a>
### Example 4:  Elimination fails

Let's make a small change to the system in the previous example and observe an example of how the elimination process can break down.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 -3x_2 -9x_3 & = & 0 
\end{align*}
$$


In [11]:
H = np.array([[1,-1,1,3],[2,-2,4,8],[3,-3,-9,0]])
H

array([[ 1, -1,  1,  3],
       [ 2, -2,  4,  8],
       [ 3, -3, -9,  0]])

In [12]:
# Add -2 times row 0 to row 1
H1 = RowAdd(H, 0, 1, -2)
print("After adding -2 times row 0 to row 1:")
print(H1, '\n')

# Add -3 times row 0 to row 2
H2 = RowAdd(H1, 0, 2, -3)
print("After adding -3 times row 0 to row 2:")
print(H2)

After adding -2 times row 0 to row 1:
[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 3. -3. -9.  0.]] 

After adding -3 times row 0 to row 2:
[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   0. -12.  -9.]]


In this case exchanging the second and third rows does not help since both rows have a zero in the middle entry.  This means that *there is no pivot* in the second column.  Let's scale the rows and look at the result.

In [13]:
# Multiply row 1 by 1/2
H3 = RowScale(H2, 1, 1.0 / 2)
print("After multiplying row 1 by 1/2:")
print(H3, '\n')

# Multiply row 2 by -1/12
H4 = RowScale(H3, 2, -1.0 / 12)
print("After multiplying row 2 by -1/12:")
print(H4)


After multiplying row 1 by 1/2:
[[  1.  -1.   1.   3.]
 [  0.   0.   1.   1.]
 [  0.   0. -12.  -9.]] 

After multiplying row 2 by -1/12:
[[ 1.   -1.    1.    3.  ]
 [ 0.    0.    1.    1.  ]
 [-0.   -0.    1.    0.75]]


When we write out the equations, we see that this system is **inconsistent**.  The last two equations contradict each other. 

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_3 & = & 0.75
\end{align*}
$$

Note that we did not make any errors in the calculation and that there is no way to reorder the equations to alleviate the problem.  This system simply does not have a solution.  We will revisit this scenario in future sections and provide a characterization for such systems.



### Example 5:  Nonunique solution

In this final example of elimination, we observe another possible outcome of the process.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
x_1 -x_2 +3x_3 & = & 5 
\end{align*}
$$


In [14]:
F = np.array([[1,-1,1,3],[2,-2,4,8],[1,-1,3,5]])
F

array([[ 1, -1,  1,  3],
       [ 2, -2,  4,  8],
       [ 1, -1,  3,  5]])

In [15]:
# Add -2 times row 0 to row 1
F1 = RowAdd(F, 0, 1, -2)
print("After adding -2 times row 0 to row 1:")
print(F1, '\n')

# Add -1 times row 0 to row 2
F2 = RowAdd(F1, 0, 2, -1)
print("After adding -1 times row 0 to row 2:")
print(F2)


After adding -2 times row 0 to row 1:
[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 1. -1.  3.  5.]] 

After adding -1 times row 0 to row 2:
[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  2.  2.]]


Similar to the previous example, we see that there is no way to bring a nonzero number into the second column of the second row.  In this case though we see that the second and third equations do not contradict each other, but are in fact the same equation.

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
2x_3 & = & 2\\
2x_3 & = & 2 
\end{align*}
$$

Let's do just two more row operations to simplify this system even further.

In [16]:
# Add -1 times row 1 to row 2
F3 = RowAdd(F2, 1, 2, -1)
print("After adding -1 times row 1 to row 2:")
print(F3, '\n')

# Multiply row 1 by 1/2
F4 = RowScale(F3, 1, 0.5)
print("After multiplying row 1 by 1/2:")
print(F4)

After adding -1 times row 1 to row 2:
[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  0.  0.]] 

After multiplying row 1 by 1/2:
[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  0.]]


Since the final equation is true for any values of $x_1$, $x_2$, $x_3$, we see that there are really only two equations to determine the values of the three unknowns.   

$$
\begin{align*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1\\
0 & = & 0 
\end{align*}
$$

Since the middle equation tells us that $x_3=1$, we can plug that value into the first equation using the idea of back substitution.  This leaves us with the single equation $x_1-x_2 = 2$.  This equation, and thus the system as a whole, has an *infinite number of solutions*.  We will revist this idea and provide further details in the next chapter. 

## Exercises

**Exercise 1:** In the system below, use a row operation functions to produce a zero coefficient in the location of the coefficient 12.  Do this first by hand and then create a NumPy array to represent the system and use the row operation functions. There are two way (from row 0 or row 2) to create the zero using $\texttt{RowAdd}$.  Try to find both.

$$
\begin{align*}
4x_1 - 2x_2 + 2x_3 & = & 7\\
5x_1 + 3x_2 + 12x_3 & = & 4\\
-9x_1 \quad\quad - 3x_3 & = & -8 
\end{align*}
$$


In [17]:
# Code solution here.
print('original matrix')
AA=np.array([[4,-2,2,7],[5,3,12,4],[-9,0,-3,-8]])
print(AA,"\n")
print('first way')
AA1=RowAdd(AA,0,1,-6)
print(AA1,"\n")
print("second way")
AA2=RowAdd(AA,2,1,4)
print(AA2,"\n")

original matrix
[[ 4 -2  2  7]
 [ 5  3 12  4]
 [-9  0 -3 -8]] 

first way
[[  4.  -2.   2.   7.]
 [-19.  15.   0. -38.]
 [ -9.   0.  -3.  -8.]] 

second way
[[  4.  -2.   2.   7.]
 [-31.   3.   0. -28.]
 [ -9.   0.  -3.  -8.]] 



**Exercise 2:** Create a NumPy array to represent the system below.  Determine which coefficient should be zero in order for the system to be upper triangular.  Use $\texttt{RowAdd}$ to carry out the row operation and then print your results.

*Note: All the diagonal elements should be 1.*

  
  $$
\begin{align*}
4x_1 + 8x_2 \, - \,\,\,\,\,  x_3 &   =   & -6\\
-2x_2   +  5x_3  &   =   & -8\\
4x_2   \,  - \,\, 9x_3 &  =  & -6 
\end{align*}
$$

In [18]:
# Code solution here.
print('original matrix')
BB=np.array([[4,8,-1,-6],[0,-2,5,-8],[0,4,-9,-6]])
print(BB,"\n")
BB1=RowAdd(BB,1,2,2)
print(BB1)
BB2=RowScale(BB1, 0, 0.25)
BB3=RowScale(BB2, 1, -0.5)
print("Final:")
print(BB3)

original matrix
[[ 4  8 -1 -6]
 [ 0 -2  5 -8]
 [ 0  4 -9 -6]] 

[[  4.   8.  -1.  -6.]
 [  0.  -2.   5.  -8.]
 [  0.   0.   1. -22.]]
Final:
[[  1.     2.    -0.25  -1.5 ]
 [ -0.     1.    -2.5    4.  ]
 [  0.     0.     1.   -22.  ]]


**Exercise 3:** Carry out the elimination process on the following system.  Define a NumPy array and make use of the row operation functions.  Print the results of each step.  Write down the upper triangular system represented by the array after all steps are completed.

*Note: All the diagonal elements should be 1.*

$$
\begin{align*}
5x_1 - 2x_2 - x_3 & = & 12\\
8x_1 - 4x_2 + 2x_3 & = & 2\\
x_1 - 2x_2 - 5x_3 & = & -8 
\end{align*}
$$

In [19]:
# Code solution here
print('original matrix')
CC=np.array([[5,-2,-1,12],[8,-4,2,2],[1,-2,-5,-8]])
print(CC,"\n")
CC1=RowAdd(CC,0,1,-1.6)
print(CC1)
CC2=RowAdd(CC1,0,2,-0.2)
print(CC2)
CC3=RowAdd(CC2,1,2,-2)
print(CC3)
CC4=RowScale(CC3,0,0.2)
print(CC4)
CC5=RowScale(CC4,1,-1/0.8)
print(CC5)
CC6=RowScale(CC5,2,-1/12)
print(CC6)

original matrix
[[ 5 -2 -1 12]
 [ 8 -4  2  2]
 [ 1 -2 -5 -8]] 

[[  5.   -2.   -1.   12. ]
 [  0.   -0.8   3.6 -17.2]
 [  1.   -2.   -5.   -8. ]]
[[  5.   -2.   -1.   12. ]
 [  0.   -0.8   3.6 -17.2]
 [  0.   -1.6  -4.8 -10.4]]
[[ 5.0000000e+00 -2.0000000e+00 -1.0000000e+00  1.2000000e+01]
 [ 0.0000000e+00 -8.0000000e-01  3.6000000e+00 -1.7200000e+01]
 [ 0.0000000e+00 -4.4408921e-16 -1.2000000e+01  2.4000000e+01]]
[[ 1.0000000e+00 -4.0000000e-01 -2.0000000e-01  2.4000000e+00]
 [ 0.0000000e+00 -8.0000000e-01  3.6000000e+00 -1.7200000e+01]
 [ 0.0000000e+00 -4.4408921e-16 -1.2000000e+01  2.4000000e+01]]
[[ 1.0000000e+00 -4.0000000e-01 -2.0000000e-01  2.4000000e+00]
 [-0.0000000e+00  1.0000000e+00 -4.5000000e+00  2.1500000e+01]
 [ 0.0000000e+00 -4.4408921e-16 -1.2000000e+01  2.4000000e+01]]
[[ 1.00000000e+00 -4.00000000e-01 -2.00000000e-01  2.40000000e+00]
 [-0.00000000e+00  1.00000000e+00 -4.50000000e+00  2.15000000e+01]
 [-0.00000000e+00  3.70074342e-17  1.00000000e+00 -2.00000000e+00]]


**Exercise 4:** Use row operations on the system below to produce an **lower triangular** system.  The first equation of the lower triangular system should contain only $x_1$ and the second equation should contain only $x_1$ and $x_2$.

*Note: All the diagonal elements should be 1.*

  $$
\begin{align*}
6x_1 + 2x_2 + 2x_3 & = & 3\\
- 4x_2 - 3x_3 & = & -12\\
2x_1 + 3x_2 + 2x_3 & = & 8
\end{align*}
$$

In [20]:
# Code solution here
print('original matrix')
DD=np.array([[6,2,2,3],[0,-4,-3,-12],[2,3,2,8]])
print(DD)
DD1=RowAdd(DD,2,1,1.5)
print(DD1)
DD1=RowAdd(DD,2,1,1.5)
print(DD1)
DD2=RowAdd(DD1,2,0,-1)
print(DD2)
DD3=RowAdd(DD2,1,0,2)
print(DD3)
DD4=RowScale(DD3, 0, 0.1)
print(DD4)
DD5=RowScale(DD4, 1, 2)
print(DD5)
DD6=RowScale(DD5, 2, 0.5)
print(DD6)


original matrix
[[  6   2   2   3]
 [  0  -4  -3 -12]
 [  2   3   2   8]]
[[6.  2.  2.  3. ]
 [3.  0.5 0.  0. ]
 [2.  3.  2.  8. ]]
[[6.  2.  2.  3. ]
 [3.  0.5 0.  0. ]
 [2.  3.  2.  8. ]]
[[ 4.  -1.   0.  -5. ]
 [ 3.   0.5  0.   0. ]
 [ 2.   3.   2.   8. ]]
[[10.   0.   0.  -5. ]
 [ 3.   0.5  0.   0. ]
 [ 2.   3.   2.   8. ]]
[[ 1.   0.   0.  -0.5]
 [ 3.   0.5  0.   0. ]
 [ 2.   3.   2.   8. ]]
[[ 1.   0.   0.  -0.5]
 [ 6.   1.   0.   0. ]
 [ 2.   3.   2.   8. ]]
[[ 1.   0.   0.  -0.5]
 [ 6.   1.   0.   0. ]
 [ 1.   1.5  1.   4. ]]


**Exercise 5:** se elimination to determine whether the following system is **consistent** or **inconsistent**.
 
$$
\begin{align*}
x_1 - x_2 - x_3 & = & 4\\
3x_1 - 3x_2 - 3x_3 & = & 12\\
5x_1 - 5x_2 - 5x_3 & = & 20 
\end{align*}
$$



In [21]:
# Code solution here
print('original matrix')
EE=np.array([[1,-1,-1,4],[3,-3,-3,12],[5,-5,-5,20]])
print(EE)
EE1=RowAdd(EE,0,1,-3)
print(EE1)
EE2=RowAdd(EE1,0,2,-5)
print(EE2)
print("It has infinity solutions, so it is consistent")

original matrix
[[ 1 -1 -1  4]
 [ 3 -3 -3 12]
 [ 5 -5 -5 20]]
[[ 1. -1. -1.  4.]
 [ 0.  0.  0.  0.]
 [ 5. -5. -5. 20.]]
[[ 1. -1. -1.  4.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
It has infinity solutions, so it is consistent


**Exercise 6:** Use elimination to show that this system of equations has no solution.

  $$
\begin{align*}
x_1  +  2x_2 +  2x_3 & = & 12\\
2x_1 +  3x_2 + 3x_3 & = & 14\\
-5x_1 - 5x_2 - 5x_3 & = & 10 
\end{align*}
$$
  


In [22]:
# Code solution here
print('original matrix')
FF=np.array([[1,2,2,12],[2,3,3,14],[-5,-5,-5,10]])
print(FF)
FF1=RowAdd(FF,0,1,-2)
print(FF1)
FF2=RowAdd(FF1,0,2,5)
print(FF2)
FF3=RowAdd(FF2,1,2,5)
print(FF3)
print("It's third line is all zero but it is 20,so there is no solution")

original matrix
[[ 1  2  2 12]
 [ 2  3  3 14]
 [-5 -5 -5 10]]
[[  1.   2.   2.  12.]
 [  0.  -1.  -1. -10.]
 [ -5.  -5.  -5.  10.]]
[[  1.   2.   2.  12.]
 [  0.  -1.  -1. -10.]
 [  0.   5.   5.  70.]]
[[  1.   2.   2.  12.]
 [  0.  -1.  -1. -10.]
 [  0.   0.   0.  20.]]
It's third line is all zero but it is 20,so there is no solution
